## Predict bin_op3
- <b>학습 모델 (gcc6_bincore_s32_h48_o3.h5) - binutils, coreutils 함수 시작점부터 32byte씩 hidden unit 48 개씩 최적화 O3 학습모델</b>
- <b>테스트 bin_op3 , stirp 파일 각 EXE 파일들</b>


In [2]:
# model load
def load_trained_model(model_path):
    model = load_model(model_path)
    print('model load 완료')
    return model

In [3]:
def eval_load(data_path, file_name, model_sequence_length):
    # data load
    print('1. test data_load start')
    data = pd.read_csv(data_path + file_name, index_col=0)
    data.reset_index(inplace=True, drop=True)
    print('2. input data load and reset_index finish')

    # input sequence 로 나눔, 맨 뒤 나머지 빼기
    rest = len(data) % (model_sequence_length)
    print('3. rest data : ', rest)
    
    # 가짜 데이터 추가
    n = model_sequence_length - rest
    k = pd.DataFrame(columns=['bin','label'], data={'bin':[0]*n,'label':['0']*n})
    data = data.append(k)
    
    # data_sub_rest = data[:len(data) - rest]
    print('4. subtracting the rest finish', data.shape)

    # one hot encoding
#    encoder = LabelEncoder()
#    encoder.classes_ = 256
#    print('encoder.classes_', encoder.classes_)
#    data_onehot = encoder.fit_transform(data['bin'])
    data_onehot = pd.get_dummies(data['bin'], columns=np.arange(256))
    
    none_column = list(set(np.arange(256)).difference(set(data_onehot.columns)))
    for j in none_column:
    #    print(j)
        data_onehot[str(j)] = 0
    
    print('5. one hot encoding finish', data_onehot.shape)
    print(type(data_onehot), data_onehot.shape)
    
    data_ndarray = data_onehot.to_numpy() # data numpy 변환
    print('6. Transforming ndarray data finish', data_ndarray.shape)
    print('7. before reshape data shape', data_ndarray.shape)
    data_reshape = data_ndarray.reshape(-1, model_sequence_length, data_ndarray.shape[1])
    print('8. after reshape data shape', data_reshape.shape)
    print('9. test file load 완료')
    
    # reshape 된 data return
    return data_reshape, data, n 

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

# 파일 path 경로
model_path = 'model/gcc6_bincore_s32_h48_o3.h5'
data_path = 'data/strip/strip_gcc6_O3/'

import os
exe_file_list=os.listdir(data_path)
print("exe_file_list:{}".format(exe_file_list),'\n')

# model load
model = load_model(model_path)

# load test
for name in exe_file_list:
    test_data, data, n = eval_load(data_path, file_name=name, model_sequence_length=32)
    print('== ', name, '파일 predict 시작 ==')
    # model 모형 체크 및 데이터 read 체크
    model.summary()
    print('test_data.shape', test_data.shape)

    # model predict
    pr = model.predict(test_data)
    print('predict 완료')

    pr_ls = list(pr.flatten())
    print('전체 predict 정답 길이', len(pr_ls))
    pr_ls = list(map(lambda x:np.round(x), pr_ls))

    # 최종결과 데이터 
    result_data = pd.DataFrame(columns=['bin','label'])
    result_data['bin'] = data['bin'][:len(data)-n]
    result_data['label'] = pr_ls[:len(data)-n]

    # 다시 가짜 데이터 제거 후 최종 csv
    print('가짜 데이터 다시 제거 후 최종 csv shape', result_data.shape)
    result_data.to_csv('predict_result/bin_op3/'+name[:-4]+'_predict_result_op3.csv')
    print(name+'파일 csv저장 완료 \n\n')

exe_file_list:['addr2line_test_str.csv', 'ar_test_str.csv', 'as-new_test_str.csv', 'bfdtest1_test_str.csv', 'bfdtest2_test_str.csv', 'chew_test_str.csv', 'cxxfilt_test_str.csv', 'elfedit_test_str.csv', 'gentestdlls_test_str.csv', 'gprof_test_str.csv', 'ld-new_test_str.csv', 'nm-new_test_str.csv', 'objcopy_test_str.csv', 'objdump_test_str.csv', 'ranlib_test_str.csv', 'readelf_test_str.csv', 'size_test_str.csv', 'strings_test_str.csv', 'strip-new_test_str.csv', 'sysinfo_test_str.csv'] 

1. test data_load start
2. input data load and reset_index finish
3. rest data :  29
4. subtracting the rest finish (1264288, 2)
5. one hot encoding finish (1264288, 256)
<class 'pandas.core.frame.DataFrame'> (1264288, 256)
6. Transforming ndarray data finish (1264288, 256)
7. before reshape data shape (1264288, 256)
8. after reshape data shape (39509, 32, 256)
9. test file load 완료
==  addr2line_test_str.csv 파일 predict 시작 ==
Model: "model_9"
________________________________________________________________

predict 완료
전체 predict 정답 길이 28064
가짜 데이터 다시 제거 후 최종 csv shape (28038, 2)
chew_test_str.csv파일 csv저장 완료 


1. test data_load start
2. input data load and reset_index finish
3. rest data :  25
4. subtracting the rest finish (1259328, 2)
5. one hot encoding finish (1259328, 256)
<class 'pandas.core.frame.DataFrame'> (1259328, 256)
6. Transforming ndarray data finish (1259328, 256)
7. before reshape data shape (1259328, 256)
8. after reshape data shape (39354, 32, 256)
9. test file load 완료
==  cxxfilt_test_str.csv 파일 predict 시작 ==
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             

predict 완료
전체 predict 정답 길이 1472640
가짜 데이터 다시 제거 후 최종 csv shape (1472637, 2)
objcopy_test_str.csv파일 csv저장 완료 


1. test data_load start
2. input data load and reset_index finish
3. rest data :  21
4. subtracting the rest finish (2492128, 2)
5. one hot encoding finish (2492128, 256)
<class 'pandas.core.frame.DataFrame'> (2492128, 256)
6. Transforming ndarray data finish (2492128, 256)
7. before reshape data shape (2492128, 256)
8. after reshape data shape (77879, 32, 256)
9. test file load 완료
==  objdump_test_str.csv 파일 predict 시작 ==
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)      

predict 완료
전체 predict 정답 길이 29568
가짜 데이터 다시 제거 후 최종 csv shape (29548, 2)
sysinfo_test_str.csv파일 csv저장 완료 


